In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [6]:
from transformers import AutoTokenizer, AutoModel
model_ckpt = "sentence-transformers/all-distilroberta-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [7]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [10]:
import pickle
embeddings_dataset = pickle.load(open('/content/all-distilroberta-v1_DocumentLevel_pkl', 'rb'))
embeddings_dataset

Dataset({
    features: ['Title', 'DocID', 'URL', 'Embeddings'],
    num_rows: 200
})

In [11]:
df = embeddings_dataset.to_pandas()
df

,Title,DocID,URL,Embeddings
0,Child Development | Psychology Today,0,https://www.psychologytoday.com/us/basics/chil...,"[-0.39214522, 0.22334796, -0.28906843, -0.2991..."
1,Parenting a Child with ADHD | Psychology Today,1,https://www.psychologytoday.com/us/basics/adhd...,"[-0.19522302, 0.48131722, 0.28389496, 0.478932..."
2,Raising an Adopted Child | Psychology Today,2,https://www.psychologytoday.com/us/basics/adop...,"[0.054309994, 0.59921056, -0.5079933, 0.022713..."
3,Parenting a Child with Autism | Psychology Today,3,https://www.psychologytoday.com/us/basics/auti...,"[0.12722898, -0.43166706, -0.25235337, -0.0036..."
4,Ending Homework Battles | Psychology Today,4,https://www.psychologytoday.com/us/blog/how-ra...,"[-0.39754033, 0.30876166, -0.063889936, -0.202..."
...,...,...,...,...
195,My Kid's Toddler Tantrums Are Getting Out of H...,195,https://www.parents.com/parenting/better-paren...,"[-0.42231005, -0.2792271, -0.32892552, -0.4109..."
196,10 Ways to Make Feeding Your Toddler 10 Times ...,196,https://www.parents.com/recipes/scoop-on-food/...,"[-0.036280636, -0.48749718, -0.2008075, 0.0973..."
197,Separation and Social Anxiety in Kids,197,https://www.parents.com/toddlers-preschoolers/...,"[-0.14739987, 0.46696678, -0.3745619, 0.032792..."
198,How Much Does My Kid Need to Eat?,198,https://www.parents.com/toddlers-preschoolers/...,"[0.012737721, -0.008351931, -0.1682226, 0.0830..."


In [12]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    #encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [13]:
question = ["fidgety child unable to sit still",
"child is forgetful in daily activities",
"child disregards rules",
"child not falling asleep in own bed",
"bedwetting in school age children",
"my child worries a lot",
"my child argues with me and screams",
"defiant argumentative behavior in child",
"potty training tips for toddlers",
"handling toddler tantrums",
"child is distracted and cannot focus",
"destructive and aggressive behavior in child"]

import pandas as pd

final_df = pd.DataFrame(columns=['Query','rank','Title','DocID','score','URL'])

result_dict={}


for i,question_embedding in enumerate(question):
  question_embedding = get_embeddings([question[i]]).cpu().detach().numpy()
  scores, samples = embeddings_dataset.get_nearest_examples("Embeddings", question_embedding, k=5)
  print("samples is->")
  print(samples)
  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  # print("------------------------")
  # print("QUERY-> "+str(i)+" "+question[i])
  # print("------------------------")
  
  query=question[i]
  rank=1
  for _, row in samples_df.iterrows():
    title=row.Title
    docID=df.loc[df['Title'] == row.Title, 'DocID'].values[0]
    qID="q_"+str(i+1)
    dID="d_"+str(docID)
    # print(f"TITLE: {row.Title}")
    # print(f"SCORE: {row.scores}")
    # print(f"DocID: {dID}")
    # print(f"URL: {row.URL}")
    # print("=" * 50)
    # print()
    if qID not in result_dict:
      result_dict[qID]={}
      result_dict[qID][dID]=row.scores
    else:
      result_dict[qID][dID]=row.scores
    final_df.loc[len(final_df)] = [query, rank, title, docID, row.scores,row.URL]

    rank+=1
#question_embedding

samples is->
{'Title': ["How to Deal With the 'Terrible Twos'", "How to Manage Your Toddler's Destructive Habits", 'Why does my 4-year old have such a hard time focusing? Should I be concerned?', 'How do I get my toddler to stop telling me "NO"?', 'How do I get my toddler to stop telling me "NO"?'], 'DocID': [175, 177, 169, 178, 13], 'URL': ['https://www.parents.com/toddlers-preschoolers/development/behavioral/tackle-terrible-twos/', 'https://www.parents.com/toddlers-preschoolers/development/behavioral/how-to-manage-your-toddlers-destructive-habits/', 'https://www.parents.com/toddlers-preschoolers/development/why-does-my-4-year-old-have-such-a-hard-time-focusing-should-i-be-concerned/', 'https://www.parents.com/toddlers-preschoolers/discipline/how-do-i-get-my-toddler-to-stop-telling-me-no/', 'https://www.parents.com/toddlers-preschoolers/discipline/how-do-i-get-my-toddler-to-stop-telling-me-no/'], 'Embeddings': [[-0.42311447858810425, 0.04573551192879677, -0.42996183037757874, -0.15704

In [14]:
print(result_dict)

{'q_1': {'d_177': 122.41325378417969, 'd_175': 122.41325378417969, 'd_169': 122.12773895263672, 'd_13': 115.3205795288086}, 'q_2': {'d_170': 102.48197937011719, 'd_169': 101.95964050292969, 'd_172': 101.94145965576172, 'd_177': 98.31216430664062, 'd_182': 91.71292877197266}, 'q_3': {'d_16': 93.65550231933594, 'd_182': 92.30066680908203, 'd_6': 91.96409606933594, 'd_177': 77.64495849609375, 'd_13': 76.74385070800781}, 'q_4': {'d_189': 120.3012924194336, 'd_188': 119.04964447021484, 'd_190': 118.13533020019531, 'd_164': 115.36119842529297, 'd_191': 111.55382537841797}, 'q_5': {'d_186': 119.17122650146484, 'd_70': 76.4955062866211, 'd_26': 76.4955062866211, 'd_27': 72.12393188476562}, 'q_6': {'d_177': 119.75677490234375, 'd_192': 111.11773681640625, 'd_9': 110.58014678955078, 'd_19': 107.02549743652344, 'd_172': 105.03428649902344}, 'q_7': {'d_14': 87.93181610107422, 'd_18': 85.86494445800781, 'd_10': 78.82499694824219, 'd_175': 78.82499694824219}, 'q_8': {'d_17': 94.11099243164062, 'd_16

In [15]:
!pip install ranx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.6/224.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyautocorpus: filename=pyautocorpus-0.1.9-cp310-cp310-linux_x86_64.whl size=189849 sha256=33ca5804d57c081e0366df7bd17ed8c71d13b824bac9acaa246a04595eb97e6f
  Stored in directory: /root/.cache/pip/wheels/e2/be/43/ec1e16468c393f6e474e3be

In [16]:
from ranx import Qrels, Run
from ranx import evaluate

In [17]:
qrels_dict = { "q_1": { "d_65":2, "d_69":2, "d_173":2, "d_1":2, "d_41":2, 
                       "d_15": 1, "d_58": 1, "d_66": 1, "d_69": 1, "d_145": 1, "d_173" : 1},
"q_2": { "d_1":2,"d_41":2,"d_65":2,"d_15":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_3": {"d_6":2,"d_17":2,"d_80":2,"d_120":2,"d_122":2,"d_178":2,
      "d_10":1,"d_13":1,"d_14":1,"d_16":1,"d_18":1,"d_19":1,"d_25":1,"d_79":1,"d_86":1,"d_182":1},
"q_4":{"d_32":2,"d_157":2,"d_159":2,"d_162":2,"d_187":2,
      "d_188":1,"d_33":1,"d_34":1,"d_37":1,"d_52":1,"d_62":1,"d_76":1,"d_109":1,"d_163":1,"d_164":1,"d_189":1},
"q_5":{"d_23":2,"d_26":2,"d_71":2,"d_98":2,"d_183":2,"d_186":2,
      "d_22":1,"d_27":1,"d_29":1,"d_49":1,"d_60":1,"d_63":1},
"q_6":{"d_5":2,"d_8":2,"d_9":2,"d_35":2,"d_50":2,"d_52":2,"d_53":2,"d_71":2,"d_90":2,"d_125":2,"d_153":2,"d_192":2,"d_193":2,"d_194":2,
      "d_34":1,"d_45":1,"d_138":1,"d_162":1,"d_168":1},
"q_7":{"d_178":2,"d_4":2,"d_10":2,"d_14":2,"d_80":2,"d_156":2,"d_195":2,
       "d_16":1,"d_19":1,"d_25":1,"d_36":1,"d_48":1,"d_70":1,"d_86":1,"d_91":1,"d_120":1,"d_152":1,"d_167":1,"d_175":1,"d_178":1,"d_179":1
      },
"q_8":{"d_4":2,"d_6":2,"d_13":2,"d_17":2,"d_19":2,"d_178":2,
      "d_18":1,"d_36":1,"d_70":1,"d_79":1},
"q_9":{"d_22":2,"d_27":2,"d_29":2,"d_49":2,"d_60":2,"d_63":2,"d_75":2,"d_84":2,"d_99":2,"d_100":2,"d_101":2,"d_102":2,"d_103":2,"d_104":2,"d_105":2,"d_149":2,"d_150":2,
      "d_23":1,"d_26":1,"d_46":1,"d_61":1,"d_71":1,"d_88":1,"d_89":1,"d_98":1},
"q_10":{"d_91":2,"d_10":2,"d_14":2,"d_195":2,"d_36":2,"d_48":2,"d_140":2,"d_152":2,"d_154":2,"d_156":2,"d_167":2,"d_179":2,"d_195":2,
        "d_16":1,"d_18":1,"d_25":1,"d_42":1,"d_70":1,"d_95":1,"d_175":1},
"q_11":{"d_41":2,"d_65":2,"d_169":2,
      "d_5":1,"d_15":1,"d_24":1,"d_50":1,"d_54":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_12":{"d_70":2,"d_85":2,"d_174":2,"d_177":2,"d_42":2,"d_96":2,
      "d_10":1,"d_16":1,"d_18":1,"d_25":1}      
 }


run_dict = result_dict


qrels = Qrels(qrels_dict)
run = Run(run_dict)

In [18]:
# Compute score for a single metric
# evaluate(qrels, run, "ndcg@5")
final_result=evaluate(qrels, run, ["map", "ndcg"])

# Compute scores for multiple metrics at once
# evaluate(qrels, run, ["map@5", "mrr"])
print(final_result)

{'map': 0.11788838750187432, 'ndcg': 0.23726661060040266}
